## Задание 1. Ошибки в исходных файлах
Что нужно сделать
У нас есть набор исходников. Необходимо выполнить git clone.


Просмотрите конфиг-файлы, dockerfile, docker-compose.yml и найдите все ошибки. Например, в dockerfile в качестве исходника для образа может не быть слоя FROM nginx:alpine/, в других dockerfile может отсутствовать volume /data, порт 6379, пользователи.


В папке web в dockerfile задайте источник python:2.7-stretch, пользователя root, порт 9000.
Подсказка: в большинстве случаев в месте, где надо что-то добавить, стоит коммент через #.
Внесите правки в docker-compose.yml:
в блок services определите сети — frontend и backend;
в блок volume добавьте путь ./web/cron/crontab_todo:/etc/crontab;
в блок consumers добавьте порт 9000;в блок redis build добавьте ./redis и порт 6379.
Когда вы подумаете, что все исправления сделаны, выполните $ docker-compose up.
Если всё сделано корректно, то запустится два экземпляра Redis.

In [ ]:
# Copyright 2024 Drengskapur
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#


Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples


In [ ]:
# @title {display-mode:"form"}
# @markdown <br/><br/><center><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/docker.svg" height="150"><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/colab.svg" height="150"></center><br/>
# @markdown <center><h1>Docker in Colab</h1></center><center>github.com/drengskapur/docker-in-colab<br/><br/><br/><b>udocker("run hello-world")</b></center><br/>
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

In [ ]:
import os
import subprocess

data_path = '/home/user/'
os.chdir(data_path)

repo_name = "docker-nginx-python2-db-redis-homework"
if not os.path.exists(repo_name):
    !git clone https://github.com/darkbenladan/docker-nginx-python2-db-redis-homework {repo_name}
os.chdir(repo_name)

Cloning into 'docker-nginx-python2-db-redis-homework'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 46 (delta 5), reused 40 (delta 2), pack-reused 0
Receiving objects: 100% (46/46), 32.71 KiB | 8.18 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
def modify_file(filepath, changes):
    """Читаем файл, вносим изменения и записываем обратно."""
    with open(filepath, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    new_lines = []
    for line in lines:
        modified = False
        for old, new in changes.items():
            if old in line:
                line = line.replace(old, new)
                modified = True
        new_lines.append(line)

    with open(filepath, 'w', encoding='utf-8') as file:
        file.writelines(new_lines)

def main():
    base_path = '/home/user/docker-nginx-python2-db-redis-homework'

    # Изменения для Dockerfile в каталоге redis
    redis_dockerfile_path = os.path.join(base_path, 'redis/Dockerfile')
    modify_file(redis_dockerfile_path, {
        'FROM #REDIS': 'FROM redis:6.2.6',
        '#PORT': 'EXPOSE 6379'
    })

    # Изменения для Dockerfile в каталоге consumers
    consumers_dockerfile_path = os.path.join(base_path, 'consumers/Dockerfile')
    modify_file(consumers_dockerfile_path, {
        'FROM python:2.7-stretch': 'FROM python:3.8-slim',
        'RUN pip install uwsgi': 'RUN pip install --upgrade pip && pip install uwsgi'
    })

    # Изменения для Dockerfile в каталоге nginx
    nginx_dockerfile_path = os.path.join(base_path, 'nginx/Dockerfile')
    modify_file(nginx_dockerfile_path, {
        'FROM python:2.7-stretch': 'FROM nginx:stable-alpine',
        '#PORT': 'EXPOSE 80 443'
    })

    # Изменения в docker-compose.yml
    docker_compose_path = os.path.join(base_path, 'docker-compose.yml')
    modify_file(docker_compose_path, {
        '#networks:': 'networks:',
        '#PORT': 'expose:'
    })

    print("Изменения успешно внесены.")

if __name__ == '__main__':
    main()

Изменения успешно внесены.


In [ ]:
udocker("--help")

user@pc$ udocker --help
Info: creating repo: /home/user/.udocker

Syntax:
  udocker  [general_options] <command>  [command_options]  <command_args>

  udocker [-h|--help|help]        :Display this help and exits
  udocker [-V|--version|version]  :Display udocker and tarball version and exits

General options common to all commands must appear before the command:
  -D, --debug                   :Debug
  -q, --quiet                   :Less verbosity
  --insecure                    :Allow insecure non authenticated https
  --repo=<directory>            :Use repository at directory
  --allow-root                  :Allow execution by root NOT recommended
  --config=<conf_file>          :Use configuration <conf_file>

Commands:
  --help [command]              :Command specific help
  showconf                      :Print all configuration options

  search <repo/expression>      :Search dockerhub for container images
  pull <repo/image:tag>         :Pull container image from dockerhub
  creat

In [ ]:
!pip install paramiko

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 17.4 MB/s eta 0:00:00


In [ ]:
import paramiko

def build_and_save_docker_image(ssh_host, ssh_user, ssh_password):
    try:
        # Создание SSH клиента
        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.connect(ssh_host, username=ssh_user, password=ssh_password)

        # Команды для сборки Docker образа и его сохранения
        docker_build_command = "docker build -t myimage:latest ."
        docker_save_command = "docker save myimage:latest -o /home/user/myimage_latest.tar"

        # Выполнение команд
        commands = [docker_build_command, docker_save_command]
        for command in commands:
            stdin, stdout, stderr = client.exec_command(command)
            print(f"Executing: {command}")
            print("STDOUT:")
            for line in stdout:
                print(line.strip())
            print("STDERR:")
            for line in stderr:
                print(line.strip())

        client.close()
    except Exception as e:
        print(f"An error occurred: {e}")

# Параметры для SSH соединения
ssh_host = "192.168.0.13"  # Предполагается, что это IP адрес, к которому нужен доступ
ssh_user = "ip172-18-0-11-coimqcqim2rg009jk400"  # SSH пользователь
ssh_password = "coimqcqi_coimqh2im2rg009jk40g"  # SSH пароль

# Запуск функции сборки и сохранения образа
build_and_save_docker_image(ssh_host, ssh_user, ssh_password)

An error occurred: [Errno 110] Connection timed out
